# DPML | Augmentation

This notebook explores how to 1) load text classifcation datasets, 2) augment those datasets. 

## Datasets

In [1]:
# shared imports
import os
import shutil
import glob
import random
import inspect
from functools import partial
from collections import defaultdict
from datasets import load_dataset, load_from_disk
import torch

# sibyl imports
import sibyl

# cleanlab imports
import numpy as np
from transformers import pipeline
from huggingface_hub import HfApi, ModelFilter
from cleanlab.filter import find_label_issues

In [2]:
dataset_configs = [
    ("imdb", "sentiment"),
    ("ag_news", "topic")
]

In [3]:
all_datasets = defaultdict(dict)
for d in dataset_configs:
    all_datasets[d[0]]["dataset"] = load_dataset(d[0])
    all_datasets[d[0]]["task_name"] = d[1]

Found cached dataset imdb (C:/Users/Fabrice/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset ag_news (C:/Users/Fabrice/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

## Augmentations

In [4]:
class Transform:
    def __init__(self, transform_class, num_outputs=1, task_name="sentiment"):
        self.transform_class = transform_class
        self.num_outputs = num_outputs
        self.task_name = task_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.intakes_target = False
        self.is_batched = False
        
        # setting class attributes
        if 'to_tense' in inspect.signature(self.transform_class).parameters:
            print("initializing class with to_tense='past'") # future & random don't work
            self.transform_instance = self.transform_class(to_tense="past")
        elif 'source_lang' in inspect.signature(self.transform_class).parameters:
            print("initializing class with source_lang='es'") 
            self.transform_instance = self.transform_class(source_lang="es")
        elif 'task_name' in inspect.signature(self.transform_class).parameters:
            print(f"initializing class with task_name='{task_name}', return_metadata=True") 
            self.transform_instance = self.transform_class(task_name=self.task_name, return_metadata=True)
        # elif isinstance(self.transform_class, LostInTranslation):
        #     self.transform_instance = self.transform_class(device=0)
        else:
            self.transform_instance = self.transform_class()
        
        # setting instance attributes
        if hasattr(self.transform_instance, "max_outputs"):
            print(f"setting max_outputs={self.num_outputs}")
            self.transform_instance.max_outputs = self.num_outputs
        if hasattr(self.transform_instance, "max_paraphrases"):
            print(f"setting max_paraphrases={self.num_outputs}")
            self.transform_instance.max_paraphrases = self.num_outputs
        if hasattr(self.transform_instance, "device"):
            if self.transform_instance.device is None or self.transform_instance.device == 'cpu':
                print(f"setting device={self.device}")
                self.transform_instance.device = self.device
        
        # selecting the transformation function
        if hasattr(self.transform_class, "generate"):
            self.transform_fn = self.transform_instance.generate
        if hasattr(self.transform_class, "augment"):
            self.transform_fn = self.transform_instance.augment
        if hasattr(self.transform_class, "transform_batch"):
            self.transform_fn = self.transform_instance.transform_batch
            self.intakes_target = True
            self.is_batched = True
            
    def synced_shuffle(self, list1, list2):
        # Shuffle two lists with same order
        # Using zip() + * operator + shuffle()
        temp = list(zip(list1, list2))
        random.shuffle(temp)
        res1, res2 = zip(*temp)
        # res1 and res2 come out as tuples, and so must be converted to lists.
        res1, res2 = list(res1), list(res2)
        return res1, res2
            
    def apply(self, texts, labels=None):
        if self.intakes_target:
            if self.is_batched:
                new_texts, new_labels = self.transform_fn((texts, labels))
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t, new_l = self.transform_fn(t, l)
                    new_texts.append(new_t)
                    new_labels.extend([new_l] * len(new_t))
        else:
            if self.is_batched:
                new_texts = self.transform_fn((texts))
                new_texts = labels
            else:
                new_texts, new_labels = [], []
                for t, l in zip(texts, labels):
                    new_t = self.transform_fn(t)
                    if len(new_t) > self.num_outputs:
                        new_t = new_t[:self.num_outputs]
                    new_texts.extend(new_t)
                    new_labels.extend([l] * len(new_t))
                    
        # print(new_texts)
        # print(new_labels)
        
        # post processing since some transformations add/remove more new outputs than expected
        if len(new_texts) == 0:
            print("no new_texts, substituting original texts...")
            new_texts = texts
        if len(new_labels) == 0:
            print("no new_labels, substituting original labels...")
            new_labels = labels
        new_texts, new_labels = self.synced_shuffle(new_texts, new_labels)
        
        expected_len = len(texts) * self.num_outputs
        new_texts = new_texts[:expected_len]
        new_labels = new_labels[:expected_len]
        
        return new_texts, new_labels

In [5]:
def augment_data(batch, transform, keep_originals=True):
    new_texts, new_labels = [], []
    for text, label in zip(batch['text'], batch['label']):
        new_text, new_label = transform.apply([text], [label])
        new_texts.extend(new_text)
        new_labels.extend(new_label)
    if keep_originals:
        return {"text": batch['text'] + new_texts, "label": batch['label'] + new_labels}
    else:
        return {"text": new_texts, "label": new_labels}

In [6]:
class CleanLabFilter:
    def __init__(self):
        self.api = HfApi()
        self.pipe = None
        self.device = 0 if torch.cuda.is_available() else -1

    def find_model_for_dataset(self, dataset_name):
        
        model_filter = ModelFilter(
            task="text-classification",
            library="pytorch",
            # model_name=dataset_name,
            trained_dataset=dataset_name)

        model_id = next(iter(self.api.list_models(filter=model_filter)))

        if model_id:
            model_id = getattr(model_id, 'modelId')
            print('Using ' + model_id + ' to support cleanlab datalabel issues.')
            self.pipe = pipeline("text-classification", 
                                 model=model_id, 
                                 device=self.device, 
                                 padding=True, 
                                 truncation=True,
                                 top_k=None)

    def extract_prediction_probabilities(self, dataset):
        output = self.pipe(dataset['text'])
        return np.stack([vectorize(o) for o in output])
    
    def extract_predition_classes(self, dataset):
        return np.argmax(self.extract_prediction_probabilities(dataset), axis=1)

    def find_num_to_remove_per_class(self, dataset, frac_to_remove=0.1):
        classes = dataset.features['label'].names
        num_classes = len(classes)
        print(num_classes)

        num_per_class = []
        for i in range(num_classes):
            class_partition = dataset.filter(lambda row: row["label"] == i)
            num_per_class.append(len(class_partition))
        num_to_remove_per_class = [int(frac_to_remove * num) for num in num_per_class]
        return num_to_remove_per_class
    
    def identify_label_issues(self, dataset, bool_mask=True):
        if self.pipe is None:
            return dataset
        
        pred_probs = self.extract_prediction_probabilities(dataset)

        label_issues = find_label_issues(
            labels=dataset['label'],
            pred_probs=pred_probs,  
            return_indices_ranked_by='self_confidence',
        )
        
        if bool_mask:
            label_issues = [1 if i in label_issues else 0 for i in range(len(dataset))]
        
        return label_issues
    
    def annotate_prediction_probabilities(self, dataset):
        pred_probs = self.extract_prediction_probabilities(dataset)
        return dataset.add_column("cleanlab_label_probabilities", pred_probs)
    
    def annotate_class_predictions(self, dataset):
        pred_classes = self.extract_predition_classes(dataset)
        return dataset.add_column("cleanlab_label_prediction", pred_classes)
    
    def annotate_label_issues(self, dataset):
        label_issues = self.identify_label_issues(dataset, bool_mask=True)
        return dataset.add_column("cleanlab_label_issue", label_issues)
    
    def annotate(self, dataset):
        if self.pipe is None:
            return dataset
        
        pred_probs = self.extract_prediction_probabilities(dataset)
        pred_classes = np.argmax(pred_probs, axis=1)
        label_issues = find_label_issues(
            labels=dataset['label'],
            pred_probs=pred_probs,  
            return_indices_ranked_by='self_confidence',
        )
        label_issues = [1 if i in label_issues else 0 for i in range(len(dataset))]
        print("adding..")
        print(pred_probs, pred_classes, label_issues)
        dataset = dataset.add_column("cleanlab_label_probabilities", pred_probs.tolist())
        dataset = dataset.add_column("cleanlab_label_prediction", pred_classes)
        dataset = dataset.add_column("cleanlab_label_issue", label_issues)
        return dataset

    def clean_dataset(self, dataset):
        label_issues = self.identify_label_issues(dataset, bool_mask=False)
        idx_to_keep = [i for i in range(len(dataset)) if i not in label_issues]
        return dataset.select(idx_to_keep)
    
def vectorize(output):
    sorted_output = sorted(output, key=lambda d: d['label']) 
    probs = np.array([d['score'] for d in sorted_output])
    return probs

### Sibyl

In [7]:
blacklist = [
    sibyl.transformations.text.emoji.emojify.Emojify,
    sibyl.transformations.text.emoji.emojify.AddPositiveEmoji,
    sibyl.transformations.text.emoji.emojify.AddNegativeEmoji,
    sibyl.transformations.text.emoji.demojify.Demojify,
    sibyl.transformations.text.emoji.demojify.RemovePositiveEmoji,
    sibyl.transformations.text.emoji.demojify.RemoveNegativeEmoji,
    sibyl.transformations.text.emoji.emojify.AddPositiveEmoji,
    sibyl.transformations.text.emoji.emojify.AddNegativeEmoji,
    sibyl.transformations.text.insertion.sentiment_phrase.InsertPositivePhrase,
    sibyl.transformations.text.insertion.sentiment_phrase.InsertNegativePhrase,
    sibyl.transformations.text.links.add_sentiment_link.AddPositiveLink,
    sibyl.transformations.text.links.add_sentiment_link.AddNegativeLink,
    sibyl.transformations.text.links.import_link_text.ImportLinkText,
    sibyl.transformations.text.negation.add_negation.AddNegation,
    sibyl.transformations.text.negation.remove_negation.RemoveNegation,
    sibyl.transformations.text.word_swap.change_synse.ChangeAntonym,
    sibyl.transformations.text.mixture.concept_mix.ConceptMix,
    sibyl.transformations.text.mixture.text_mix.TextMix,
    sibyl.transformations.text.mixture.text_mix.SentMix,
    sibyl.transformations.text.mixture.text_mix.WordMix,
    sibyl.transformations.text.generative.concept2sentence.Concept2Sentence
]
sibyl_transforms = [t for t in sibyl.TRANSFORMATIONS if t not in blacklist]

In [ ]:
# control variables
save_dir       = "./datasets/"
dataset_size   = 1000
batch_size     = 10
keep_originals = False
seed           = 130
frameworkd_id  = "sibyl"
current_dataset_name = ""

cl_filter = CleanLabFilter()

# augment
for dataset_name, dataset_data in all_datasets.items():
    
    num_samples = '.' + str(dataset_size) if dataset_size > 0 else ''
    kot = 'KOT' if keep_originals else 'KOF'
    
    # load dataset
    dataset = dataset_data['dataset']['train'].shuffle(seed=seed)
    if dataset_size > 0:
        dataset = dataset.select(range(dataset_size))
        
    if current_dataset_name != dataset_name:
        cl_filter.find_model_for_dataset(dataset_name)
        current_dataset_name = dataset_name
        
    # annotations for original dataset
    print("saving original dataset with annotations.")
    save_path = os.path.join(save_dir, f"{dataset_name}.orig{num_samples}")
    dataset = cl_filter.annotate(dataset)
    dataset.save_to_disk(save_path)
    
    columns_to_remove = ['cleanlab_label_probabilities', 'cleanlab_label_prediction', 'cleanlab_label_issue']
    for c in columns_to_remove:
        if c in dataset.features:
            dataset = dataset.remove_columns(c)
        
    for t in sibyl_transforms:

        t_name = t.__name__
        save_path = os.path.join(save_dir, 
            f"{dataset_name}.{frameworkd_id}.{t_name}{num_samples}.{kot}")

        if os.path.exists(save_path):
            print(f"Existing dataset found save at {save_path}")
        else:
            print(f"Processing {t_name} to save at {save_path}")
            
            # augment
            aug = partial(augment_data, 
                          transform=Transform(t, task_name=dataset_data['task_name']), 
                          keep_originals=keep_originals)
            aug_dataset = dataset.map(aug, batched=True, batch_size=batch_size)
            
            # annotations for augmented dataset
            aug_dataset = cl_filter.annotate(aug_dataset)
            
            # save to path
            aug_dataset.save_to_disk(save_path)

Loading cached shuffled indices for dataset at C:/Users/Fabrice/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-f4dc5a5f07ef8007.arrow


Using XSY/albert-base-v2-imdb-calssification to support cleanlab datalabel issues.
saving original dataset with annotations.
adding..
[[0.00825037 0.99174964]
 [0.99631846 0.00368147]
 [0.94703943 0.05296055]
 ...
 [0.00342701 0.99657297]
 [0.00376336 0.99623662]
 [0.99421185 0.00578811]] [1 0 0 1 0 0 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 1 0 1
 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0
 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0
 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0
 1 1 0 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 1
 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0
 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0
 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 1 0 0
 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0
 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing ExpandContractions to save at ./datasets/imdb.sibyl.ExpandContractions.1000.KOF
initializing class with task_name='sentiment', return_metadata=True


  0%|          | 0/100 [00:00<?, ?ba/s]

...

  0%|          | 0/100 [00:00<?, ?ba/s]

adding..
[[0.04121456 0.95878547]
 [0.99342877 0.00657124]
 [0.94445926 0.05554071]
 ...
 [0.00402293 0.9959771 ]
 [0.00580176 0.9941982 ]
 [0.99332047 0.00667952]] [1 0 0 1 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 1 1 1
 1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0
 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0
 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0
 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1
 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0
 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1
 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 1 1 1 0
 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 0 1 0 0
 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 0 1 0
 1 1 1 0 1

  0%|          | 0/100 [00:00<?, ?ba/s]

adding..
[[0.97472376 0.02527624]
 [0.98730147 0.01269852]
 [0.98341137 0.01658864]
 ...
 [0.98192167 0.01807828]
 [0.87029678 0.12970322]
 [0.9795838  0.02041622]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0

  0%|          | 0/100 [00:00<?, ?ba/s]